In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from ipywidgets import interact, Dropdown
from statsmodels.tsa.seasonal import seasonal_decompose
# from scipy.stats import zscore, skew, normaltest

In [2]:
df = pd.read_csv('../Data/3mon.csv')
df.head()

,id,created_datetime,water_meter_code,flow_rate,flow_volume,flow_today,net_total,created_by,creation_date,modified_date
0,5799676,2025-04-01 00:00:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05
1,5799677,2025-04-01 00:15:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05
2,5799678,2025-04-01 00:30:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05
3,5799679,2025-04-01 00:45:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05
4,5799680,2025-04-01 01:00:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05


In [3]:
len(df)

1088132

In [4]:
df = df.rename(columns={'id':'Id','created_datetime':'LocalTimeCol','water_meter_code':'MeterCode','flow_rate':'FR','flow_volume':'FV','flow_today':'Today', 'net_total':'NetTotal'})
df.head()

,Id,LocalTimeCol,MeterCode,FR,FV,Today,NetTotal,created_by,creation_date,modified_date
0,5799676,2025-04-01 00:00:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05
1,5799677,2025-04-01 00:15:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05
2,5799678,2025-04-01 00:30:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05
3,5799679,2025-04-01 00:45:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05
4,5799680,2025-04-01 01:00:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05


In [5]:
df['LocalTimeCol'] = pd.to_datetime(df['LocalTimeCol'])
df['Date'] = df['LocalTimeCol'].dt.date
df['Time'] = df['LocalTimeCol'].dt.time
df.head()

,Id,LocalTimeCol,MeterCode,FR,FV,Today,NetTotal,created_by,creation_date,modified_date,Date,Time
0,5799676,2025-04-01 00:00:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05,2025-04-01,00:00:00
1,5799677,2025-04-01 00:15:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05,2025-04-01,00:15:00
2,5799678,2025-04-01 00:30:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05,2025-04-01,00:30:00
3,5799679,2025-04-01 00:45:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05,2025-04-01,00:45:00
4,5799680,2025-04-01 01:00:00,BFM 1,0.0,0.0,0.0,0.0,SYNC_APP,2025-06-10 10:05:41,2025-06-10 15:35:05,2025-04-01,01:00:00


In [6]:
df.MeterCode.nunique()

161

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088132 entries, 0 to 1088131
Data columns (total 12 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Id             1088132 non-null  int64         
 1   LocalTimeCol   1088132 non-null  datetime64[ns]
 2   MeterCode      1088132 non-null  object        
 3   FR             1071126 non-null  float64       
 4   FV             174064 non-null   float64       
 5   Today          1069275 non-null  float64       
 6   NetTotal       1070097 non-null  float64       
 7   created_by     1088132 non-null  object        
 8   creation_date  1088132 non-null  object        
 9   modified_date  1088132 non-null  object        
 10  Date           1088132 non-null  object        
 11  Time           1088132 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(6)
memory usage: 99.6+ MB


In [8]:
df[["FR", "FV", "NetTotal", "Today"]] = (
    df.groupby('MeterCode')[["FR", "FV", "NetTotal", "Today"]]
    .transform(lambda x: x.interpolate(method='linear').bfill().ffill())
)

In [9]:
df[["FR", "FV", "NetTotal", "Today"]] = df[["FR", "FV", "NetTotal", "Today"]].fillna(0)

In [10]:
df.isna().sum()

Id               0
LocalTimeCol     0
MeterCode        0
FR               0
FV               0
Today            0
NetTotal         0
created_by       0
creation_date    0
modified_date    0
Date             0
Time             0
dtype: int64

In [11]:
df = df.sort_values(['LocalTimeCol','MeterCode'])
df.head()

,Id,LocalTimeCol,MeterCode,FR,FV,Today,NetTotal,created_by,creation_date,modified_date,Date,Time
1475,5801151,2025-04-01,AMR-04,0.0,0.0,-61.730000,-782.070007,SYNC_APP,2025-06-10 10:05:49,2025-06-10 15:35:12,2025-04-01,00:00:00
1534,5801210,2025-04-01,AMR-05,0.0,0.0,0.000000,-782.070007,SYNC_APP,2025-06-10 10:05:49,2025-06-10 15:35:12,2025-04-01,00:00:00
1593,5801269,2025-04-01,AMR-08,0.0,0.0,25.570000,1152.479980,SYNC_APP,2025-06-10 10:05:49,2025-06-10 15:35:12,2025-04-01,00:00:00
1652,5801328,2025-04-01,AMR-10,0.0,0.0,37.580002,2217.760010,SYNC_APP,2025-06-10 10:05:49,2025-06-10 15:35:12,2025-04-01,00:00:00
3776,5803452,2025-04-01,AMR-101,0.0,0.0,0.000000,0.330000,SYNC_APP,2025-06-10 10:06:04,2025-06-10 15:35:27,2025-04-01,00:00:00


In [12]:
df.to_csv("../Data/Data_Cleaned.csv")

In [13]:
data = df.copy()

In [14]:
df.set_index('LocalTimeCol', inplace=True)
df_hour = (df.groupby('MeterCode').resample('h')['FV','FR','Today'].mean().reset_index())
df_hour.head()

,MeterCode,LocalTimeCol,FV,FR,Today
0,AMR-04,2025-04-01 00:00:00,0.0,0.0,-15.4325
1,AMR-04,2025-04-01 01:00:00,0.0,0.0,0.0000
2,AMR-04,2025-04-01 02:00:00,0.0,0.0,0.0000
3,AMR-04,2025-04-01 03:00:00,0.0,0.0,0.0000
4,AMR-04,2025-04-01 04:00:00,0.0,0.0,0.0000


In [15]:
df_day = (df.groupby('MeterCode').resample('D')['FV','FR','Today'].mean().reset_index())
df_day.head()

,MeterCode,LocalTimeCol,FV,FR,Today
0,AMR-04,2025-04-01,0.0,-1.521458,-27.563125
1,AMR-04,2025-04-02,0.0,-2.176979,-22.466354
2,AMR-04,2025-04-03,0.0,-1.397812,-25.335416
3,AMR-04,2025-04-04,0.0,-0.935625,-16.968750
4,AMR-04,2025-04-05,0.0,-3.341667,-43.894272


In [16]:
df_day.isna().sum()

MeterCode         0
LocalTimeCol      0
FV              548
FR              548
Today           548
dtype: int64

In [17]:
# df_hour.to_excel('Data/Hour_data.xlsx', index=False)
# df_day.to_excel('Data/Day_data.xlsx', index=False)

In [18]:
# BoxPlot for hourly chart for analyse variation

def plot_boxplot_fr(meter_code):
    plt.figure(figsize=(16, 6))

    data = df_hour[df_hour['MeterCode'] == meter_code].copy()
    data['Date'] = pd.to_datetime(data['LocalTimeCol']).dt.date

    # 2 rows, 1 column, first plot
    plt.subplot(2, 1, 1)
    sns.boxplot(x='Date', y='FR', data=data, hue='Date', legend=False)
    plt.title(f'Hourly FR Distribution for MeterCode: {meter_code}')
    plt.xlabel('Date')
    plt.ylabel('FR')
    plt.xticks(rotation=45) 
    plt.grid(True, alpha=0.4)

    plt.subplot(2, 1, 2)
    sns.boxplot(x='Date', y='FV', data=data, hue='Date', legend=False)
    plt.title(f'Hourly FV Distribution for MeterCode: {meter_code}')
    plt.xlabel('Date')
    plt.ylabel('FV')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.4)

    plt.tight_layout()
    plt.show()


# Create interactive dropdown
interact(plot_boxplot_fr, meter_code=Dropdown(options=[str(m) for m in df_hour['MeterCode'].unique()], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_boxplot_fr(meter_code)>

In [19]:

# BoxPlot for daily chart for analyse variation
def plot_boxplot(meter_code):
    plt.figure(figsize=(16, 6))
    data = df_day[df_day['MeterCode'] == meter_code]
    
    plt.subplot(2, 1, 1)
    sns.boxplot(x='LocalTimeCol', y='FR', data=data, hue='LocalTimeCol', legend=False)
    plt.title(f'Daily FR Distribution for MeterCode: {meter_code}')
    plt.xlabel('Date')
    plt.ylabel('FR')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.4)

    plt.subplot(2, 1, 2)
    sns.boxplot(x='LocalTimeCol', y='FV', data=data, hue='LocalTimeCol', legend=False)
    plt.title(f'Daily FV Distribution for MeterCode: {meter_code}')
    plt.xlabel('Date')
    plt.ylabel('FV')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.4)

    plt.tight_layout()
    plt.show()


# Create interactive dropdown
interact(plot_boxplot, meter_code=Dropdown(options=[str(m) for m in df_hour['MeterCode'].unique()], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_boxplot(meter_code)>

In [20]:
# HistPlot for daily chart 
def plot_boxplot_fr(meter_code):
    plt.figure(figsize=(16, 6))
    data = df_day[df_day['MeterCode'] == meter_code]
    
    plt.subplot(2, 1, 1)
    sns.histplot(data['FR'], bins=5, kde=True, color='skyblue')
    plt.title('Histogram - Hourly FR Distribution')
    plt.xlabel('FR Value')
    plt.ylabel('Frequency')
    # plt.xticks(rotation=45)
    plt.grid(True, alpha=0.4)

    plt.subplot(2, 1, 2)
    sns.histplot(data['FV'], bins=5, kde=True, color='orange')
    plt.title('Histogram - Hourly FV Distribution')
    plt.xlabel('FV Value')
    plt.ylabel('Frequency')
    # plt.xticks(rotation=45)
    plt.grid(True, alpha=0.4)

    plt.tight_layout()
    plt.show()


# Create interactive dropdown
interact(plot_boxplot_fr, meter_code=Dropdown(options=[str(m) for m in df_hour['MeterCode'].unique()], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_boxplot_fr(meter_code)>

In [21]:


# HistPlot for daily chart 
def plot_boxplot_fr(meter_code):
    plt.figure(figsize=(16, 6))
    data = df_day[df_day['MeterCode'] == meter_code]
    
    plt.subplot(2, 1, 1)
    sns.histplot(data['FR'], bins=5, kde=True, color='skyblue')
    plt.title('Histogram - Daily FR Distribution')
    plt.xlabel('FR Value')
    plt.ylabel('Frequency')
    # plt.xticks(rotation=45)
    plt.grid(True, alpha=0.4)

    plt.subplot(2, 1, 2)
    sns.histplot(data['FV'], bins=5, kde=True, color='orange')
    plt.title('Histogram - Daily FV Distribution')
    plt.xlabel('FV Value')
    plt.ylabel('Frequency')
    # plt.xticks(rotation=45)
    plt.grid(True, alpha=0.4)

    plt.tight_layout()
    plt.show()


# Create interactive dropdown
interact(plot_boxplot_fr, meter_code=Dropdown(options=[str(m) for m in df_hour['MeterCode'].unique()], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_boxplot_fr(meter_code)>

In [22]:
df_day['LocalTimeCol'] = pd.to_datetime(df_day['LocalTimeCol'])

# Get unique MeterCode values
meter_codes = df_day['MeterCode'].unique()
print(f"Number of unique MeterCode values: {len(meter_codes)}")

Number of unique MeterCode values: 161


In [23]:
# Define plotting function
def plot_fv(meter_code):
    plt.figure(figsize=(16, 6))
    meter_data = df_day[df_day['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    plt.plot(meter_data['LocalTimeCol'], meter_data['FV'], color='blue', label=f'FV - {meter_code}')
    plt.title(f'Daily FV for MeterCode: {meter_code}')
    plt.xlabel('Date')
    plt.ylabel('FV Value')
    plt.legend()
    plt.xlim(meter_data['LocalTimeCol'].min(), meter_data['LocalTimeCol'].max())
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=2))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.6)
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fv, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fv(meter_code)>

In [24]:
# Define plotting function
def plot_fv(meter_code):
    plt.figure(figsize=(16, 6))
    meter_data = df_day[df_day['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    plt.plot(meter_data['LocalTimeCol'], meter_data['FR'], color='blue', label=f'FR - {meter_code}')
    plt.title(f'Daily FR for MeterCode: {meter_code}')
    plt.xlabel('Date')
    plt.ylabel('FR Value')
    plt.legend()
    plt.xlim(meter_data['LocalTimeCol'].min(), meter_data['LocalTimeCol'].max())
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=2))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.6)
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fv, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fv(meter_code)>

####

In [ ]:
# Define plotting function
def plot_fv(meter_code):
    plt.figure(figsize=(16, 6))
    meter_data = df_hour[df_hour['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    plt.plot(meter_data['LocalTimeCol'], meter_data['FV'], color='red', label=f'FV - {meter_code}')
    plt.title(f'Hourly FV for MeterCode: {meter_code}')
    plt.xlabel('Time')
    plt.ylabel('FV Value')
    plt.legend()
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    plt.xlim(meter_data['LocalTimeCol'].min(), meter_data['LocalTimeCol'].max())
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=96))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fv, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fv(meter_code)>

In [ ]:
# Define plotting function
def plot_fv(meter_code):
    plt.figure(figsize=(16, 6))
    meter_data = df_day[df_day['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    plt.plot(meter_data['LocalTimeCol'], meter_data['FV'], color='red', label=f'FV - {meter_code}')
    plt.title(f'Daily FV for MeterCode: {meter_code}')
    plt.xlabel('Time')
    plt.ylabel('FV Value')
    plt.legend()
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    plt.xlim(meter_data['LocalTimeCol'].min(), meter_data['LocalTimeCol'].max())
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=96))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fv, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fv(meter_code)>

In [73]:
# Define plotting function
def plot_fv(meter_code):
    plt.figure(figsize=(16, 6))
    meter_data = df_day[df_day['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    plt.plot(meter_data['LocalTimeCol'], meter_data['FR'], color='blue', label=f'FR - {meter_code}')
    plt.title(f'Daily FR for MeterCode: {meter_code}')
    plt.xlabel('Time')
    plt.ylabel('FR Value')
    plt.legend()
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    plt.xlim(meter_data['LocalTimeCol'].min(), meter_data['LocalTimeCol'].max())
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=96))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fv, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fv(meter_code)>

In [ ]:
# Define plotting function
def plot_fr(meter_code):
    plt.figure(figsize=(16, 6))
    meter_data = df_hour[df_hour['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    plt.plot(meter_data['LocalTimeCol'], meter_data['FR'], color='blue', label=f'FR - {meter_code}')
    plt.title(f'Hourly FR for MeterCode: {meter_code}')
    plt.xlabel('Time')
    plt.ylabel('FR Value')
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    plt.legend()
    plt.xlim(meter_data['LocalTimeCol'].min(), meter_data['LocalTimeCol'].max())
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=96))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fr, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fr(meter_code)>

In [71]:
# Define plotting function
def plot_fr(meter_code):
    plt.figure(figsize=(16, 6))
    meter_data = df_day[df_day['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    plt.plot(meter_data['LocalTimeCol'], meter_data['FR'], color='blue', label=f'FR - {meter_code}')
    plt.title(f'Daily FR for MeterCode: {meter_code}')
    plt.xlabel('Time')
    plt.ylabel('FR Value')
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    plt.legend()
    plt.xlim(meter_data['LocalTimeCol'].min(), meter_data['LocalTimeCol'].max())
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=96))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fr, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fr(meter_code)>

In [27]:
df.reset_index('LocalTimeCol', inplace=True)
df.head()

,LocalTimeCol,Id,MeterCode,FR,FV,Today,NetTotal,created_by,creation_date,modified_date,Date,Time
0,2025-04-01,5801151,AMR-04,0.0,0.0,-61.730000,-782.070007,SYNC_APP,2025-06-10 10:05:49,2025-06-10 15:35:12,2025-04-01,00:00:00
1,2025-04-01,5801210,AMR-05,0.0,0.0,0.000000,-782.070007,SYNC_APP,2025-06-10 10:05:49,2025-06-10 15:35:12,2025-04-01,00:00:00
2,2025-04-01,5801269,AMR-08,0.0,0.0,25.570000,1152.479980,SYNC_APP,2025-06-10 10:05:49,2025-06-10 15:35:12,2025-04-01,00:00:00
3,2025-04-01,5801328,AMR-10,0.0,0.0,37.580002,2217.760010,SYNC_APP,2025-06-10 10:05:49,2025-06-10 15:35:12,2025-04-01,00:00:00
4,2025-04-01,5803452,AMR-101,0.0,0.0,0.000000,0.330000,SYNC_APP,2025-06-10 10:06:04,2025-06-10 15:35:27,2025-04-01,00:00:00


In [54]:
# df_day.isna().sum()
df_day.fillna(0, inplace=True)
df_day.isna().sum()

MeterCode       0
LocalTimeCol    0
FV              0
FR              0
Today           0
dtype: int64

In [ ]:
# Perform seasonal decomposition
print("seasonal decomposition\n")

# Define plotting function for seasonal decomposition
def plot_fv_decomposition(meter_code):
    
    meter_data = df_day[df_day['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    # print(meter_data)
    if len(meter_data) < 6:
        print(f"Warning: Insufficient data for MeterCode {meter_code} (need at least 48 hours)")
        return
    
    meter_data = meter_data.set_index('LocalTimeCol')
    
    # Perform seasonal decomposition
    try:
        decomposition = seasonal_decompose(meter_data['FV'], model='additive', period=24)
    except ValueError as e:
        print(f"Error for MeterCode {meter_code}: {e}")
        return
    
    observed = decomposition.observed
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    
    # Create subplots for decomposition components
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
    
    # Plot observed
    ax1.plot(observed.index, observed, color='blue')
    ax1.set_title(f'FV Decomposition for MeterCode: {meter_code} - Observed')
    ax1.set_ylabel('Observed')
    ax1.grid(False)
    
    # Plot trend
    ax2.plot(trend.index, trend, color='blue')
    ax2.set_title('Trend')
    ax2.set_ylabel('Trend')
    ax2.grid(False)
    
    # Plot seasonal
    ax3.plot(seasonal.index, seasonal, color='blue')
    ax3.set_title('Seasonal')
    ax3.set_ylabel('Seasonal')
    ax3.grid(False)
    
    # Plot residual
    ax4.plot(residual.index, residual, color='blue')
    ax4.set_title('Residual')
    ax4.set_ylabel('Residual')
    ax4.grid(False)
    
    # Format x-axis
    ax4.xaxis.set_major_locator(mdates.DayLocator(interval=4))
    ax4.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    
    # Adjust layout
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fv_decomposition, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

seasonal decomposition



interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fv_decomposition(meter_code)>

In [58]:
# Perform seasonal decomposition
print("seasonal decomposition\n")

# Define plotting function for seasonal decomposition
def plot_fv_decomposition(meter_code):
    
    meter_data = df_day[df_day['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    # print(meter_data)
    if len(meter_data) < 6:
        print(f"Warning: Insufficient data for MeterCode {meter_code} (need at least 48 hours)")
        return
    
    meter_data = meter_data.set_index('LocalTimeCol')
    
    # Perform seasonal decomposition
    try:
        decomposition = seasonal_decompose(meter_data['FR'], model='additive', period=24)
    except ValueError as e:
        print(f"Error for MeterCode {meter_code}: {e}")
        return
    
    observed = decomposition.observed
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    
    # Create subplots for decomposition components
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
    
    # Plot observed
    ax1.plot(observed.index, observed, color='blue')
    ax1.set_title(f'FR Decomposition for MeterCode: {meter_code} - Observed')
    ax1.set_ylabel('Observed')
    ax1.grid(False)
    
    # Plot trend
    ax2.plot(trend.index, trend, color='blue')
    ax2.set_title('Trend')
    ax2.set_ylabel('Trend')
    ax2.grid(False)
    
    # Plot seasonal
    ax3.plot(seasonal.index, seasonal, color='blue')
    ax3.set_title('Seasonal')
    ax3.set_ylabel('Seasonal')
    ax3.grid(False)
    
    # Plot residual
    ax4.plot(residual.index, residual, color='blue')
    ax4.set_title('Residual')
    ax4.set_ylabel('Residual')
    ax4.grid(False)
    
    # Format x-axis
    ax4.xaxis.set_major_locator(mdates.DayLocator(interval=4))
    ax4.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    
    # Adjust layout
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fv_decomposition, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

seasonal decomposition



interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fv_decomposition(meter_code)>

In [30]:
# A rolling mean (also called a moving average) is a technique used to smooth out short-term fluctuations and highlight 
# longer-term trends or cycles in time series data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from ipywidgets import interact, Dropdown

# Assuming 'df' is your DataFrame and it's already loaded
# df['LocalTimeCol'] = pd.to_datetime(df['LocalTimeCol']) 
# meter_codes = df['MeterCode'].unique()

# Define plotting function for FV with rolling means
def plot_fv_rolling_mean(meter_code):
    # Filter data for the selected MeterCode
    meter_data = df_hour[df_hour['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    
    # --- ADD THIS BLOCK to filter for the first month ---
    if not meter_data.empty:
        # Get the first timestamp for the selected meter
        start_date = meter_data['LocalTimeCol'].min()
        # Calculate the end date as one month after the start date
        end_date = start_date + pd.DateOffset(months=1)
        # Filter the DataFrame to this one-month period
        meter_data = meter_data[meter_data['LocalTimeCol'] < end_date]
    # --- END OF ADDED BLOCK ---

    # Ensure sufficient data for rolling means
    if len(meter_data) < 6:
        print(f"Warning: Insufficient data for MeterCode {meter_code} (need at least 6 hours)")
        return
    
    # Compute rolling means
    meter_data['fv_rolling_6h'] = meter_data['FV'].rolling(window=6, min_periods=1).mean()
    meter_data['fv_rolling_12h'] = meter_data['FV'].rolling(window=12, min_periods=1).mean()
    
    # Create the plot
    plt.figure(figsize=(16, 6))
    plt.plot(meter_data['LocalTimeCol'], meter_data['FV'], label="Actual FV", color='blue', alpha=0.8)
    plt.plot(meter_data['LocalTimeCol'], meter_data['fv_rolling_6h'], label="6-Hour Rolling Mean", color='red', alpha=0.8)
    plt.plot(meter_data['LocalTimeCol'], meter_data['fv_rolling_12h'], label="12-Hour Rolling Mean", color='green', alpha=0.8)
    
    # Set title and labels
    plt.title(f"First Month of Hourly Flow Volume with Rolling Mean for MeterCode: {meter_code}")
    plt.xlabel("Time")
    plt.ylabel("Flow Volume")
    plt.legend()
    
    # Format x-axis
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=2)) # Adjusted interval for better visibility
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    
    # Add grid
    plt.grid(True, linestyle='--', alpha=0.4)
    
    # Adjust layout
    plt.tight_layout()
    plt.show()

# Create interactive dropdown (this part remains the same)
interact(plot_fv_rolling_mean, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fv_rolling_mean(meter_code)>

In [ ]:
# Define plotting function for FV with rolling means
def plot_fv_rolling_mean(meter_code):
    # Filter data for the selected MeterCode
    meter_data = df[df['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    
    # Ensure sufficient data for rolling means
    if len(meter_data) < 6:
        print(f"Warning: Insufficient data for MeterCode {meter_code} (need at least 6 hours)")
        return
    
    # Compute rolling means
    meter_data['fv_rolling_6h'] = meter_data['FV'].rolling(window=6, min_periods=1).mean()
    meter_data['fv_rolling_12h'] = meter_data['FV'].rolling(window=12, min_periods=1).mean()
    
    # Create the plot
    plt.figure(figsize=(16, 6))
    plt.plot(meter_data['LocalTimeCol'], meter_data['FV'], label="Actual FV", color='blue', alpha=0.8)
    plt.plot(meter_data['LocalTimeCol'], meter_data['fv_rolling_6h'], label="6-Hour Rolling Mean", color='red', alpha=0.8)
    plt.plot(meter_data['LocalTimeCol'], meter_data['fv_rolling_12h'], label="12-Hour Rolling Mean", color='green', alpha=0.8)
    
    # Set title and labels
    plt.title(f"Hourly Flow Volume with Rolling Mean for MeterCode: {meter_code}")
    plt.xlabel("Time")
    plt.ylabel("Flow Volume")
    plt.legend()
    
    # Format x-axis: one tick per day
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=4))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    
    # Add grid
    plt.grid(True, linestyle='--', alpha=0.4)
    
    # Adjust layout
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fv_rolling_mean, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fv_rolling_mean(meter_code)>

In [ ]:
# Define plotting function for FV with rolling means
def plot_fv_rolling_mean(meter_code):
    # Filter data for the selected MeterCode
    meter_data = df_day[df_day['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    
    # Ensure sufficient data for rolling means
    if len(meter_data) < 6:
        print(f"Warning: Insufficient data for MeterCode {meter_code} (need at least 6 hours)")
        return
    
    # Compute rolling means
    meter_data['fv_rolling_6h'] = meter_data['FV'].rolling(window=6, min_periods=1).mean()
    meter_data['fv_rolling_12h'] = meter_data['FV'].rolling(window=12, min_periods=1).mean()
    
    # Create the plot
    plt.figure(figsize=(16, 6))
    plt.plot(meter_data['LocalTimeCol'], meter_data['FV'], label="Actual FV", color='blue', alpha=0.8)
    plt.plot(meter_data['LocalTimeCol'], meter_data['fv_rolling_6h'], label="6-Hour Rolling Mean", color='red', alpha=0.8)
    plt.plot(meter_data['LocalTimeCol'], meter_data['fv_rolling_12h'], label="12-Hour Rolling Mean", color='green', alpha=0.8)
    
    # Set title and labels
    plt.title(f"Daily Flow Volume with Rolling Mean for MeterCode: {meter_code}")
    plt.xlabel("Time")
    plt.ylabel("Flow Volume")
    plt.legend()
    
    # Format x-axis: one tick per day
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=4))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    
    # Add grid
    plt.grid(True, linestyle='--', alpha=0.4)
    
    # Adjust layout
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fv_rolling_mean, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fv_rolling_mean(meter_code)>

In [69]:
df_day.head()

,MeterCode,LocalTimeCol,FV,FR,Today
0,AMR-04,2025-04-01,0.0,-1.521458,-27.563125
1,AMR-04,2025-04-02,0.0,-2.176979,-22.466354
2,AMR-04,2025-04-03,0.0,-1.397812,-25.335416
3,AMR-04,2025-04-04,0.0,-0.935625,-16.968750
4,AMR-04,2025-04-05,0.0,-3.341667,-43.894272


In [70]:
# Define plotting function for FR with rolling means
def plot_fr_rolling_mean(meter_code):
    # Filter data for the selected MeterCode
    meter_data = df_day[df_day['MeterCode'] == meter_code].sort_values('LocalTimeCol')
    
    # Ensure sufficient data for rolling means
    if len(meter_data) < 6:
        print(f"Warning: Insufficient data for MeterCode {meter_code} (need at least 6 hours)")
        return
    
    # Compute rolling means
    meter_data['fr_rolling_6h'] = meter_data['FR'].rolling(window=6, min_periods=1).mean()
    meter_data['fr_rolling_12h'] = meter_data['FR'].rolling(window=12, min_periods=1).mean()
    
    # Create the plot
    plt.figure(figsize=(16, 6))
    plt.plot(meter_data['LocalTimeCol'], meter_data['FR'], label="Actual FR", color='blue', alpha=0.8)
    plt.plot(meter_data['LocalTimeCol'], meter_data['fr_rolling_6h'], label="6-Hour Rolling Mean", color='red', alpha=0.8)
    plt.plot(meter_data['LocalTimeCol'], meter_data['fr_rolling_12h'], label="12-Hour Rolling Mean", color='green', alpha=0.8)

    # Set title and labels
    plt.title(f"Daily Flow Rate with Rolling Mean for MeterCode: {meter_code}")
    plt.xlabel("Time")
    plt.ylabel("Flow Rate")
    plt.legend()
    
    # Format x-axis: one tick per day
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=4))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    
    # Add grid
    plt.grid(True, linestyle='--', alpha=0.4)
    
    # Adjust layout
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fr_rolling_mean, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fr_rolling_mean(meter_code)>

In [32]:
df_hour['Hour'] = df_hour['LocalTimeCol'].dt.hour

hourly_avg = df_hour.groupby(['MeterCode', 'Hour'])['FV'].mean().reset_index()

morning_hours = range(6, 12)  # 6:00 AM to 11:59 AM
night_hours = range(18, 24)   # 6:00 PM to 11:59 PM

peak_times = []

# Find peak morning and night hours for each MeterCode
for meter in df_hour['MeterCode'].unique():
    meter_data = hourly_avg[hourly_avg['MeterCode'] == meter]
    
    # Morning peak
    morning_data = meter_data[meter_data['Hour'].isin(morning_hours)]
    morning_peak_hour = morning_data.loc[morning_data['FV'].idxmax(), 'Hour'] if not morning_data.empty else None
    morning_peak_value = morning_data['FV'].max() if not morning_data.empty else None
    
    # Night peak
    night_data = meter_data[meter_data['Hour'].isin(night_hours)]
    night_peak_hour = night_data.loc[night_data['FV'].idxmax(), 'Hour'] if not night_data.empty else None
    night_peak_value = night_data['FV'].max() if not night_data.empty else None
    
    peak_times.append({
        'MeterCode': meter,
        'Morning_Peak_Hour': morning_peak_hour,
        'Morning_Peak_FV': morning_peak_value,
        'Night_Peak_Hour': night_peak_hour,
        'Night_Peak_FV': night_peak_value
    })

# Create DataFrame of peak times
peak_df = pd.DataFrame(peak_times)
print("Peak Water Usage Times:")
peak_df.head()



Peak Water Usage Times:


,MeterCode,Morning_Peak_Hour,Morning_Peak_FV,Night_Peak_Hour,Night_Peak_FV
0,AMR-04,6,0.0,18,0.0
1,AMR-05,6,0.0,18,0.0
2,AMR-08,6,0.0,18,0.0
3,AMR-10,6,0.0,18,0.0
4,AMR-101,6,0.0,18,0.0


In [ ]:
# Define plotting function
def plot_fv_peaks(meter_code):
   
    meter_data = hourly_avg[hourly_avg['MeterCode'] == meter_code]
    
    # Compute 4-hour rolling mean of hourly averages
    meter_data = meter_data.sort_values('Hour')
    meter_data['FV_Rolling_4h'] = meter_data['FV'].rolling(window=4, min_periods=1, center=True).mean()
    
    # Get peak hours for this MeterCode
    meter_peaks = peak_df[peak_df['MeterCode'] == meter_code]
    morning_peak = meter_peaks['Morning_Peak_Hour'].iloc[0] if not meter_peaks['Morning_Peak_Hour'].isna().iloc[0] else None
    night_peak = meter_peaks['Night_Peak_Hour'].iloc[0] if not meter_peaks['Night_Peak_Hour'].isna().iloc[0] else None
    
    # Create plot
    plt.figure(figsize=(12, 4))
    plt.plot(meter_data['Hour'], meter_data['FV'], label='Average FV', color='blue', alpha=0.4)
    plt.plot(meter_data['Hour'], meter_data['FV_Rolling_4h'], label='4-Hour Rolling Mean', color='red', alpha=0.6)
    
    # Highlight peak hours
    if morning_peak is not None:
        plt.axvline(x=morning_peak, color='green', linestyle='--', label='Morning Peak', alpha=0.5)
    if night_peak is not None:
        plt.axvline(x=night_peak, color='purple', linestyle='--', label='Night Peak', alpha=0.5)
    
    # Set title and labels
    plt.title(f'Hourly Water Usage (FV) for MeterCode: {meter_code}')
    plt.xlabel('Hour of Day')
    plt.ylabel('Average Flow Volume')
    plt.legend()
    
    # Format x-axis
    plt.xticks(range(0, 24))
    # plt.subplots_adjust(left=1, right=0.9, top=2, bottom=0.1)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(plot_fv_peaks, meter_code=Dropdown(options=[str(m) for m in df_hour['MeterCode'].unique()], description='MeterCode:'))

interactive(children=(Dropdown(description='MeterCode:', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AMR…

<function __main__.plot_fv_peaks(meter_code)>

##### Forecasting with Prophet (week -4)

In [34]:
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [35]:
# Function to fit Prophet model and forecast
def prophet_forecast(data, freq='h', forecast_periods=168):
    # Initialize the Prophet model
    model = Prophet(
        yearly_seasonality=False,  # No yearly pattern in short dataset
        weekly_seasonality=True,   # Weekly patterns (weekdays vs. weekends)
        daily_seasonality=True,    # Daily patterns (morning vs. night)
        seasonality_mode='additive'  # Additive model for water usage
    )
    # Fit the model
    model.fit(data)
    
    # Create future dataframe
    if freq == 'h':
        future = model.make_future_dataframe(periods=forecast_periods, freq='h')
    else:
        future = model.make_future_dataframe(periods=forecast_periods, freq='D')
    
    # Forecast
    forecast = model.predict(future)
    return model, forecast

In [36]:
# Prepare data and fit Prophet model for each MeterCode
h_forecasts = {}
h_models = {}

for meter in df_hour['MeterCode'].unique():
   
    meter_data = df_hour[df_hour['MeterCode'] == meter][['LocalTimeCol', 'FV']].copy()
    meter_data = meter_data.rename(columns={'LocalTimeCol': 'ds', 'FV': 'y'})
    
    # Check for sufficient data
    if len(meter_data) < 2:
        print(f"Skipping MeterCode {meter}: Insufficient data for Prophet")
        continue
    
    # Fit Prophet model and forecast
    try:
        model, forecast = prophet_forecast(meter_data, freq='h', forecast_periods=168)
        h_models[meter] = model
        h_forecasts[meter] = forecast
    except Exception as e:
        print(f"Error for MeterCode {meter}: {e}")
        continue

print(f"\nProphet models fitted for {len(h_forecasts)} MeterCode values")

22:05:25 - cmdstanpy - INFO - Chain [1] start processing
22:05:25 - cmdstanpy - INFO - Chain [1] done processing
22:05:26 - cmdstanpy - INFO - Chain [1] start processing
22:05:26 - cmdstanpy - INFO - Chain [1] done processing
22:05:27 - cmdstanpy - INFO - Chain [1] start processing
22:05:27 - cmdstanpy - INFO - Chain [1] done processing
22:05:29 - cmdstanpy - INFO - Chain [1] start processing
22:05:30 - cmdstanpy - INFO - Chain [1] done processing
22:05:31 - cmdstanpy - INFO - Chain [1] start processing
22:05:32 - cmdstanpy - INFO - Chain [1] done processing
22:05:33 - cmdstanpy - INFO - Chain [1] start processing
22:05:33 - cmdstanpy - INFO - Chain [1] done processing
22:05:35 - cmdstanpy - INFO - Chain [1] start processing
22:05:36 - cmdstanpy - INFO - Chain [1] done processing
22:05:38 - cmdstanpy - INFO - Chain [1] start processing
22:05:39 - cmdstanpy - INFO - Chain [1] done processing
22:05:40 - cmdstanpy - INFO - Chain [1] start processing
22:05:40 - cmdstanpy - INFO - Chain [1]


Prophet models fitted for 161 MeterCode values


In [37]:
# Define plotting function for forecasts
def plot_forecast(meter_code):
    
    meter_data = df_hour[df_hour['MeterCode'] == meter_code][['LocalTimeCol', 'FV']].copy()
    meter_data = meter_data.rename(columns={'LocalTimeCol': 'ds', 'FV': 'y'})
    h_forecast = h_forecasts.get(meter_code)
    
    if h_forecast is None:
        print(f"No forecast available for MeterCode {meter_code}")
        return
    
    plt.figure(figsize=(12, 6))
    
    plt.plot(meter_data['ds'], meter_data['y'], label='Historical FV', color='blue', alpha=0.4)
    plt.plot(h_forecast['ds'], h_forecast['yhat'], label='Forecast', color='red', alpha=0.6)
    
    # Plot uncertainty intervals - future values lies in these area
    if 'yhat_lower' in h_forecast and 'yhat_upper' in h_forecast:  
        plt.fill_between(h_forecast['ds'], h_forecast['yhat_lower'], h_forecast['yhat_upper'], 
                         color='green', alpha=0.2, label='Uncertainty Interval')
    
    # Set title and labels
    plt.title(f'Hourly FV: Actual vs. Forecast for MeterCode: {meter_code}', pad=20)
    plt.xlabel('Date', labelpad=20)
    plt.ylabel('Flow Volume', labelpad=20)
    
    # Position legend below the plot
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.5), ncol=3)
    
    # Format x-axis
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=2))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    
    plt.grid(True, linestyle='--', alpha=0.8)
    
    plt.tight_layout()
    plt.show()


# Create interactive dropdown
interact(plot_forecast, meter_code=Dropdown(options=[str(m) for m in df_hour['MeterCode'].unique()], description='MeterCode: '))

interactive(children=(Dropdown(description='MeterCode: ', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AM…

<function __main__.plot_forecast(meter_code)>

In [38]:
def hourly_plot_components(meter_code):
    # Get the model and forecast for the selected MeterCode
    h_model = h_models[meter_code]
    h_forecast = h_forecasts[meter_code]
    
    # Plot components
    
    h_model.plot_components(h_forecast,figsize=(12, 6))
    plt.suptitle(f'Prophet Forecast Components for MeterCode: {meter_code}')
    plt.tight_layout()
    plt.show()

# Create interactive dropdown
interact(hourly_plot_components, meter_code=Dropdown(options=[str(m) for m in df_hour['MeterCode'].unique()], description='MeterCode: '))

interactive(children=(Dropdown(description='MeterCode: ', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AM…

<function __main__.hourly_plot_components(meter_code)>

In [39]:
d_forecasts = {}
d_models = {}

for meter in df_day['MeterCode'].unique():
    meter_data = df_day[df_day['MeterCode'] == meter][['LocalTimeCol', 'FV']].copy()
    meter_data['LocalTimeCol'] = pd.to_datetime(meter_data['LocalTimeCol'])
    meter_data.dropna(subset=['FV'], inplace=True)
    meter_data = meter_data.rename(columns={'LocalTimeCol': 'ds', 'FV': 'y'})

    if len(meter_data) < 10:
        print(f"Skipping MeterCode {meter}: Insufficient data for Prophet")
        continue

    try:
        d_model, d_forecast = prophet_forecast(meter_data, freq='D', forecast_periods=7)
        d_models[meter] = d_model
        d_forecasts[meter] = d_forecast
    except Exception as e:
        print(f"Error for MeterCode {meter}: {e}")
        continue

print(f"\n Prophet models fitted for {len(d_forecasts)} MeterCode values")

22:06:46 - cmdstanpy - INFO - Chain [1] start processing
22:06:46 - cmdstanpy - INFO - Chain [1] done processing
22:06:46 - cmdstanpy - INFO - Chain [1] start processing
22:06:47 - cmdstanpy - INFO - Chain [1] done processing
22:06:47 - cmdstanpy - INFO - Chain [1] start processing
22:06:49 - cmdstanpy - INFO - Chain [1] done processing
22:06:50 - cmdstanpy - INFO - Chain [1] start processing
22:06:52 - cmdstanpy - INFO - Chain [1] done processing
22:06:53 - cmdstanpy - INFO - Chain [1] start processing
22:06:54 - cmdstanpy - INFO - Chain [1] done processing
22:06:55 - cmdstanpy - INFO - Chain [1] start processing
22:06:55 - cmdstanpy - INFO - Chain [1] done processing
22:06:57 - cmdstanpy - INFO - Chain [1] start processing
22:06:58 - cmdstanpy - INFO - Chain [1] done processing
22:06:59 - cmdstanpy - INFO - Chain [1] start processing
22:07:00 - cmdstanpy - INFO - Chain [1] done processing
22:07:00 - cmdstanpy - INFO - Chain [1] start processing
22:07:01 - cmdstanpy - INFO - Chain [1]


 Prophet models fitted for 161 MeterCode values


In [40]:
def daily_plot_forecast(meter_code):
    meter_data = df_day[df_day['MeterCode'] == meter_code][['LocalTimeCol', 'FV']].copy()
    meter_data = meter_data.rename(columns={'LocalTimeCol': 'ds', 'FV': 'y'})

    forecast_data = d_forecasts.get(meter_code)

    if forecast_data is None:
        print(f"No forecast available for MeterCode {meter_code}")
        return

    plt.figure(figsize=(12, 5))

    plt.plot(meter_data['ds'], meter_data['y'], label='Historical FV', color='blue', alpha=0.6)
    plt.plot(forecast_data['ds'], forecast_data['yhat'], label='Forecast', color='red', alpha=0.8)

    if 'yhat_lower' in forecast_data and 'yhat_upper' in forecast_data:
        plt.fill_between(forecast_data['ds'], forecast_data['yhat_lower'], forecast_data['yhat_upper'],
                         color='green', alpha=0.2, label='Uncertainty Interval')

    plt.title(f'Daily FV: Actual vs Forecast for MeterCode: {meter_code}', fontsize=14)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Flow Volume', fontsize=12)

    plt.xticks(rotation=45, fontsize=10)
    plt.yticks(fontsize=10)

    # Use major ticks only for clarity
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=2))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.4), ncol=3)
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()


# Create interactive dropdown
interact(daily_plot_forecast, meter_code=Dropdown(options=[str(m) for m in df_day['MeterCode'].unique()], description='MeterCode: '))

interactive(children=(Dropdown(description='MeterCode: ', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AM…

<function __main__.daily_plot_forecast(meter_code)>

In [41]:
def daily_plot_components(meter_code):
    # Get the model and forecast for the selected MeterCode
    d_model = d_models[meter_code]
    d_forecast = d_forecasts[meter_code]
    
    # Plot components
    
    d_model.plot_components(d_forecast,figsize=(12, 6))
    plt.suptitle(f'Prophet Forecast Components for MeterCode: {meter_code}')
    plt.tight_layout()
    plt.xticks(rotation=45)
    plt.show()



# Create interactive dropdown
interact(daily_plot_components, meter_code=Dropdown(options=[str(m) for m in df_day['MeterCode'].unique()], description='MeterCode: '))

interactive(children=(Dropdown(description='MeterCode: ', options=('AMR-04', 'AMR-05', 'AMR-08', 'AMR-10', 'AM…

<function __main__.daily_plot_components(meter_code)>

In [42]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [43]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_forecast(actual, predicted):

    # Calculate standard metrics
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    
    # Create a mask to select elements where the actual value is not zero
    mask = actual != 0
    
    # Check if there are any non-zero actuals to avoid division by zero
    if not np.any(mask):
        mape = np.nan # Return NaN if all actuals are zero, as MAPE is undefined
    else:
        # Calculate MAPE only for the non-zero elements
        mape = np.mean(np.abs((actual[mask] - predicted[mask]) / actual[mask])) * 100

    return rmse, mae, mape

In [44]:
meter_codes = df['MeterCode'].unique()

In [45]:
# Calculate hourly metrics for each MeterCode
hour_metrics = {}
for meter in meter_codes:
    actual = df_hour[df_hour['MeterCode'] == meter]['FV']
    forecast = h_forecasts[meter]['yhat'].head(len(actual))  
    rmse, mae, mape = evaluate_forecast(actual, forecast)
    hour_metrics[meter] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

ValueError: Input contains NaN.

In [ ]:
# Summary table
metrics_df = pd.DataFrame.from_dict(hour_metrics, orient='index')
print("\nSummary of Evaluation hourly Metrics Across All MeterCodes:")
print(metrics_df.describe())

In [ ]:
# Define plotting and metrics display function
def plot_and_evaluate(meter_code):

    # Display metrics
    metric_values = hour_metrics[meter_code]
    print(f"Evaluation Hourly Metrics for MeterCode: {meter_code}")
    print(f"RMSE: {metric_values['RMSE']:.4f}")
    print(f"MAE: {metric_values['MAE']:.4f}")
    print(f"MAPE: {metric_values['MAPE']:.2f}%")

# Create interactive dropdown
interact(plot_and_evaluate, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

In [ ]:
# df_fv_daily = df_hour.groupby([pd.Grouper(key='LocalTimeCol', freq='D'), 'MeterCode'])['FV'].sum().reset_index()

In [ ]:
# # Calculate daily metrics for each MeterCode
daily_metrics = {}
for meter in meter_codes:
    actual = df_day[df_day['MeterCode'] == meter]['FV']
    forecast = d_forecasts.get(meter, pd.DataFrame())
    if forecast.empty:
        print(f"No forecast for {meter}, skipping...")
        continue
    merged = pd.merge(df_day[df_day['MeterCode'] == meter][['LocalTimeCol', 'FV']], 
                      forecast.rename(columns={'ds': 'LocalTimeCol'}), 
                      on='LocalTimeCol', how='left')
    actual = merged['FV']
    predicted = merged['yhat'].fillna(0)
    if len(actual) == 0 or len(predicted) == 0:
        print(f"No matching data for {meter}, skipping...")
        continue
    rmse, mae, mape = evaluate_forecast(actual, predicted)
    daily_metrics[meter] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

In [ ]:
# Summary table
metrics_df = pd.DataFrame.from_dict(daily_metrics, orient='index')
print("\n Summary of Evaluation Metrics Across All MeterCodes:")
print(metrics_df.describe())

In [ ]:
# Define daily plotting and metrics display function
def plot_and_evaluate(meter_code):

    # Display metrics
    metric_values = daily_metrics[meter_code]
    print(f"Evaluation Daily Metrics for MeterCode: {meter_code}")
    print(f"RMSE: {metric_values['RMSE']:.4f}")
    print(f"MAE: {metric_values['MAE']:.4f}")
    print(f"MAPE: {metric_values['MAPE']:.2f}%")

# Create interactive dropdown
interact(plot_and_evaluate, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

In [ ]:
# Split at a chosen date
split_date = '2025-05-01'

In [ ]:
# train_data = df_hour[df_hour['LocalTimeCol'] <= split_date].copy()
# test_data = df_hour[df_hour['LocalTimeCol'] > split_date].copy()

In [ ]:
# Function to calculate evaluation metrics
def evaluate_forecast_for_split_date(actual, predicted):

    actual = np.array(actual, dtype=float)
    predicted = np.array(predicted, dtype=float)
    
     # Compute errors
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)

    # Mask to avoid division by zero in MAPE
    mask = (actual != 0) & (~np.isnan(actual))
    if mask.sum() > 0:
        mape = np.mean(np.abs((actual[mask] - predicted[mask]) / actual[mask])) * 100
    else:
        mape = np.nan

    return rmse, mae, mape

In [ ]:
def split_and_forecast(train_data, test_data, freq='h'):
    model = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=True,
        daily_seasonality=True,
        seasonality_mode='additive'
    )

    model.fit(train_data)
    
    # Create future dataframe
    if freq == 'h':
        future = model.make_future_dataframe(periods=len(test_data),freq=freq)
    else:
        future = model.make_future_dataframe(periods=len(test_data),freq=freq)
    
    forecast = model.predict(future)

    # Extract predictions for the test period
    forecasted_test = forecast.tail(len(test_data))
    return model, forecast, forecasted_test

In [ ]:
# Prepare data and fit Prophet model for each MeterCode
hr_forecasts = {}
hr_forecast_test = {}
hr_models = {}
hr_metrics = {}

for meter in df_hour['MeterCode'].unique():
   
    meter_data = df_hour[df_hour['MeterCode'] == meter].copy()

    train_data = meter_data[meter_data['LocalTimeCol'] <= split_date].copy()
    test_data = meter_data[meter_data['LocalTimeCol'] > split_date].copy()

    train_meter_fv_data = train_data[train_data['MeterCode'] == meter][['LocalTimeCol', 'FV']].copy()
    train_meter_fv_data = train_meter_fv_data.rename(columns={'LocalTimeCol': 'ds', 'FV': 'y'})

    test_meter_fv_data = test_data[test_data['MeterCode'] == meter][['LocalTimeCol', 'FV']].copy()
    test_meter_fv_data = test_meter_fv_data.rename(columns={'LocalTimeCol': 'ds', 'FV': 'y'})
    
    # Check for sufficient data
    if train_meter_fv_data.empty or len(train_meter_fv_data) < 2 or test_meter_fv_data.empty:
        print(f"Skipping MeterCode {meter}: Insufficient data for Prophet")
        continue
    
    # Fit Prophet model and forecast
    try:
        model, forecast, forecast_test = split_and_forecast(train_meter_fv_data, test_meter_fv_data, freq='h')
        hr_models[meter] = model
        hr_forecasts[meter] = forecast
        hr_forecast_test[meter] = forecast_test
    except Exception as e:
        print(f"Error for MeterCode {meter}: {e}")
        continue


    # Check alignment
    if len(forecast_test) != len(test_meter_fv_data):
        print(f"Length mismatch for MeterCode {meter}: forecast_test={len(forecast_test)}, test_data={len(test_meter_fv_data)}")

    # Evaluate metrics
    actual = test_meter_fv_data['y']
    predicted = forecast_test['yhat']
    # print(f"predicted length for MeterCode {meter}: {len(predicted)}")
    if len(actual) > 0 and len(predicted) > 0:
        rmse, mae, mape = evaluate_forecast_for_split_date(actual, predicted)
        hr_metrics[meter] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

print(f"\nProphet models fitted for {len(hr_forecasts)} MeterCode values")

In [ ]:
# Define daily plotting and metrics display function
def plot_and_evaluate(meter_code):

    # Display metrics
    metric_values = hr_metrics[meter_code]
    print(f"Evaluation hourly Metrics based on split date for MeterCode: {meter_code}")
    print(f"RMSE: {metric_values['RMSE']:.4f}")
    print(f"MAE: {metric_values['MAE']:.4f}")
    print(f"MAPE: {metric_values['MAPE']:.2f}%")

# Create interactive dropdown
interact(plot_and_evaluate, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

In [ ]:
# Prepare data and fit Prophet model for each MeterCode
dy_forecasts = {}
dy_forecast_test = {}
dy_models = {}
dy_metrics = {}

for meter in df_day['MeterCode'].unique():
   
    meter_data = df_day[df_day['MeterCode'] == meter].copy()

    train_data = meter_data[meter_data['LocalTimeCol'] <= split_date].copy()
    test_data = meter_data[meter_data['LocalTimeCol'] > split_date].copy()

    train_meter_fv_data = train_data[train_data['MeterCode'] == meter][['LocalTimeCol', 'FV']].copy()
    train_meter_fv_data = train_meter_fv_data.rename(columns={'LocalTimeCol': 'ds', 'FV': 'y'})

    test_meter_fv_data = test_data[test_data['MeterCode'] == meter][['LocalTimeCol', 'FV']].copy()
    test_meter_fv_data = test_meter_fv_data.rename(columns={'LocalTimeCol': 'ds', 'FV': 'y'})
    
    # Check for sufficient data
    if train_meter_fv_data.empty or len(train_meter_fv_data) < 2 or test_meter_fv_data.empty:
        print(f"Skipping MeterCode {meter}: Insufficient data for Prophet")
        continue
    
    # Fit Prophet model and forecast
    try:
        model, forecast, forecast_test = split_and_forecast(train_meter_fv_data, test_meter_fv_data, freq='h')
        dy_models[meter] = model
        dy_forecasts[meter] = forecast
        dy_forecast_test[meter] = forecast_test
    except Exception as e:
        print(f"Error for MeterCode {meter}: {e}")
        continue


    # Check alignment
    if len(forecast_test) != len(test_meter_fv_data):
        print(f"Length mismatch for MeterCode {meter}: forecast_test={len(forecast_test)}, test_data={len(test_meter_fv_data)}")

    # Evaluate metrics
    actual = test_meter_fv_data['y']
    predicted = forecast_test['yhat']
    # print(f"predicted length for MeterCode {meter}: {len(predicted)}")
    if len(actual) > 0 and len(predicted) > 0:
        rmse, mae, mape = evaluate_forecast_for_split_date(actual, predicted)
        dy_metrics[meter] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

print(f"\nProphet models fitted for {len(hr_forecasts)} MeterCode values")

In [ ]:
# Define daily plotting and metrics display function
def plot_and_evaluate(meter_code):

    # Display metrics
    metric_values = dy_metrics[meter_code]
    print(f"Evaluation daily Metrics based on split date for MeterCode: {meter_code}")
    print(f"RMSE: {metric_values['RMSE']:.4f}")
    print(f"MAE: {metric_values['MAE']:.4f}")
    print(f"MAPE: {metric_values['MAPE']:.2f}%")

# Create interactive dropdown
interact(plot_and_evaluate, meter_code=Dropdown(options=[str(m) for m in meter_codes], description='MeterCode:'))

In [ ]:
df_hour

In [ ]:
df_hour.MeterCode.value_counts()

In [ ]:
# Custom function to train, forecast, and evaluate with tuning
def train_forecast_evaluate_tuned(train_data, test_data, freq='h', changepoint_prior_scale=0.05, holidays=None):
    model = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=True,
        daily_seasonality=True,
        seasonality_mode='additive',
        changepoint_prior_scale=changepoint_prior_scale,
        holidays=holidays
    )
   
    model.fit(train_data)
    if freq == 'h':
        future = model.make_future_dataframe(periods=len(test_data), freq=freq)
    else:
        future = model.make_future_dataframe(periods=len(test_data), freq=freq)
    
    forecast = model.predict(future)
    forecast_test = forecast.tail(len(test_data))
    
    actual = test_data['y']
    predicted = forecast_test['yhat']
    if len(actual) > 0 and len(predicted) > 0:
        rmse, mae, mape = evaluate_forecast_for_split_date(actual, predicted)
    else:
        rmse, mae, mape = np.nan, np.nan, np.nan
    
    return model, forecast, forecast_test, rmse, mae, mape

In [ ]:
# Define holidays
holidays = pd.DataFrame({
    'holiday': ['labour day', 'shutdown'],
    'ds': pd.to_datetime(['2025-05-01',  # Labor Day
                          '2025-05-10'  # shutdown
                        ]), 
    'lower_window': 0,
    'upper_window': 0
})

In [ ]:
# Ensure LocalTimeCol is in datetime format
df_hour['LocalTimeCol'] = pd.to_datetime(df_hour['LocalTimeCol'])

In [ ]:

# Identify meters with mostly zero values
zero_proportions = df_hour.groupby('MeterCode')['FV'].apply(lambda x: (x == 0).mean()).reset_index(name='zero_proportion')
meters_with_high_zeros = zero_proportions[zero_proportions['zero_proportion'] > 0.8]

In [ ]:
zero_proportions.head()

In [ ]:
# Identify meters with insufficient data
start_time = df_hour['LocalTimeCol'].min()
end_time = df_hour['LocalTimeCol'].max()
expected_hours = pd.date_range(start=start_time, end=end_time, freq='h').size
print(f"Expected hours based on data range: {expected_hours} (from {start_time} to {end_time})")
timestamp_coverage = df_hour.groupby('MeterCode').agg(
    total_points=('FV', 'count'),
    unique_timestamps=('LocalTimeCol', 'nunique')
).reset_index()
timestamp_coverage['coverage_percentage'] = (timestamp_coverage['unique_timestamps'] / expected_hours) * 100
meters_with_insufficient_data = timestamp_coverage[timestamp_coverage['coverage_percentage'] < 50]

# Combine problematic meters
problematic_meters = set(meters_with_high_zeros['MeterCode']).union(set(meters_with_insufficient_data['MeterCode']))
print(f"Total problematic MeterCode values to exclude: {len(problematic_meters)}")
print(f"Problematic MeterCodes: {problematic_meters}")

# Filter df_hour to exclude problematic meters
df_clean = df_hour[~df_hour['MeterCode'].isin(problematic_meters)].copy()
print(f"Remaining MeterCode values after exclusion: {df_clean['MeterCode'].nunique()}")
